### RAG 시나리오(문서)
1. 시나리오1: LLM에 직접 질문&답.
2. 시나리오2(RAG): DB 데이터의 벡터검색 결과를 LLM이 참조하는 답변.
3. 2개의 시나리오 결과를 비교해보세요 

### 0. 파이썬 라이브러리 로딩 및 오라클 DB 접속

In [ ]:
# 파이썬 라이브러리 로딩 및 오라클 DB 접속
import oracledb,oml
import re, os, time
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor,get_db_version,exec_sql,exec_sql_rows,dml_sql,ddl_sql,clean_text

# DB 접속
conn, cursor = get_conn_cursor("vector","vector","localhost:1521/freepdb1")

# DB 버전 확인
get_db_version(cursor)

### 1. RAG DB 데이터 준비
1. 문서->텍스트 추출->청킹->임베딩 실행
2. 임베딩 정보를 테이블에 저장, 벡터 인덱싱 생성

In [ ]:
# 벡터 테이블 정리
dr_tbl = """truncate table rag_tbl_v """
ddl_sql(cursor,dr_tbl)
print("테이블이 정리 됐습니다")

# 문서에서 텍스트 추출 및 청킹
# 임베딩 및 인덱싱
print("임베딩 및 인덱싱을 진행중입니다 ......")
cr_tbl = """
insert into rag_tbl_v  
SELECT dt.id AS 문서번호, dt.doc_name as 문서명, et.embed_id as 청크ID , et.embed_data AS 문서내용, to_vector(et.embed_vector) as 벡터
  FROM rag_tbl dt, 
         dbms_vector_chain.utl_to_embeddings(
         dbms_vector_chain.utl_to_chunks(
           dbms_vector_chain.utl_to_text(dt.doc),
           JSON('{"by" : "words","max" : "150", "overlap" : "20","split" : "newline","language" : "korean","normalize" : "all"}')),
		   JSON('{"provider":"database", "model":"PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2"}')) t,
       JSON_TABLE(t.column_value, '$'columns (embed_id number,embed_data varchar2(4000), embed_vector CLOB)) et
WHERE dt.id = 3
"""
start_time = time.time()
rows = ddl_sql(cursor,cr_tbl)
execute_time = time.time() - start_time
print("임베딩 완료 시간 :",execute_time)

# 벡터 인덱싱 정리
ddl_sql(cursor,"drop index if exists rag_tbl_v_idx")

# 벡터 인덱싱
crIdx = """ create vector index rag_tbl_v_idx on rag_tbl_v(벡터)
ORGANIZATION INMEMORY NEIGHBOR GRAPH WITH DISTANCE COSINE
"""
ddl_sql(cursor,crIdx)
print("인덱싱 완료")
print("벡터 데이터 준비가 됐습니다.")

### 공통질문

In [ ]:
# 공통 질문
user_q = "공공기관 분리발주 대상 소프트웨어는 어떤것들이 있나요?"
print("공통질문 :",user_q)

### 2. LLM 질의 답변 경우
- LLM : LLAMA3.2
- 질문은 별도 자료 제공없이 질문합니다.

In [ ]:
# LLM(llama3.2) 질의 응답
# Oracle's API for ollama
gen_param = """{"provider":"ollama", "host":"local", "url":"http://localhost:11434/api/generate", "model":"llama3.2"}"""

gen_q = """select dbms_vector.utl_to_generate_text(:txt, json(:param)) from dual"""
print("LLAMA3.2에 질문 : ",user_q," 에 대한 답변을 준비중입니다......")

import time
start_time = time.time()
result = exec_sql_rows(cursor,gen_q,{"param":gen_param,"txt":user_q})
execute_time = time.time() - start_time
lob_result = result[0][0]
print("LLAMA3.2가 학습된 정보를 기초로 답변한 내용은 다음과 같습니다.")
print("---------------------------------------------------------------------")
print(lob_result.read())
print("---------------------------------------------------------------------")
print("실행 시간 :",execute_time)

### 참고 : 시나리오에 사용할 문서

In [ ]:
# pdf 문서(소프트웨어 분리발주 매뉴얼)디스플레이
import os
from IPython.display import FileLink
img_dir = "./data/doc/rag/"  
img_file = "소프트웨어_분리발주_매뉴얼_개정배포_16.7.pdf" 
img_path = os.path.join(img_dir, img_file)

display(FileLink(img_path))

### 3. RAG 실행 경우
- RAG 실행 순서는 1. 사용자 질의가 들어 있는 문장에 대한 DB 벡터 검색, 2. 벡터 검색 결과를 참조하는 프롬프트 생성, 3. 프로프트를 LLM에 전달, 답변 요청, 4. LLM 답변
- LLM : LLAMA3.2

In [ ]:
# 사용자 지정 문서를 참조한 RAG 답변

print(user_q,"에 대한 답변을 DB문서를 참조해서 답변을 준비중입니다......")

# 1. vector search
sim_search = """
select T.문서명,T.청크ID, T.문서내용
from rag_tbl_v T
order by vector_distance(T.벡터,dbms_vector.utl_to_embedding(:u_sentence, 
                                                       json('{"provider":"database", "model":"PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2"}')),cosine)
fetch first 3 rows only
"""
simq_start_time = time.time()
sim_result = exec_sql(cursor,sim_search,{"u_sentence":user_q})
sim_result = clean_text(sim_result)
sim_result['문서내용']=sim_result['문서내용'].str.replace('�', '', regex=False)
simq_time = time.time() - simq_start_time

print("벡터쿼리 실행 시간 ",simq_time)

# ollama API by oracle
gen_ollama_params = """{"provider":"ollama", 
                          "host":"local", 
                          "url":"http://180.68.194.221:11434/api/generate",
                          "model":"llama3.2"}
                          """
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

user_question = user_q

## 2. 프롬프트 생성 
template = """
Context 정보만를 이용해서 답변해주세요.
'답변'과'참조문서'로 구분해서 답변해주세요.답변 작성시 친숙한 어조로 작성해주세요.
'답변'은 500자 이내로 요약 정리해주세요. '참조문서'에는 context의 '문서명'과 '청크ID'를 명시해주세요.
CONTEXT : {context}
[INST]QUESTION : {question}[/INST]
"""
print("프로프트 \n", template)

prompt = ChatPromptTemplate.from_template(template)
print("langchain-ollama 프롬프트를 생성하였고, LLAMA3.2가 답변을 생성중입니다.......")

# 3. LLM 답변 생성
model = OllamaLLM(model="llama3.2", max_new_tokens= 2000 ,temperature=0.1, base_url='http://180.68.194.221:11434')
chain = prompt | model 

rag_start_time = time.time()
answer = chain.invoke({"context":sim_result,"question": user_question})
rag_time = time.time() - rag_start_time

print("LLM 답변 생성시간 : ",rag_time)
print("-----------------------------------------------------------------------------------")
print(answer)
print("-----------------------------------------------------------------------------------")

### 참고 : 벡터 검색 결과

In [ ]:
# 벡터 검색 결과
display(
    sim_result.head(10).style
    .set_properties(**{'text-align': 'left'})
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
)

시나리오 종료

--------------------------